<a href="https://colab.research.google.com/github/rajasreekalli/Data-Visualization/blob/main/First_Modell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('/content/drive/MyDrive/insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [4]:
# 1. Define features (X) and target (y)
X = df.drop(columns=['charges'])
y = df['charges']

In [5]:
# Check missing values
print(df.isna().sum().sum(), 'missing values')

0 missing values


In [6]:
# 2. Train test split the data to prepare for machine learning
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
# 3. Identify each feature as numerical, ordinal, or nominal.
# Ans: In my point of view age, bmi, children are ordinal. May be these columns also consider numeric. Smoker, sex, region are nominal.

In [8]:
# 4. One Hot Encode the nominal features

In [9]:
cat_selector = make_column_selector(dtype_include='object')

In [10]:
cat_data = X_train[cat_selector(X_train)]
cat_data

,sex,smoker,region
693,male,no,northwest
1297,female,no,southeast
634,male,no,southwest
1022,male,yes,southeast
178,female,no,southwest
...,...,...,...
1095,female,no,northeast
1130,female,no,southeast
1294,male,no,northeast
860,female,yes,southwest


In [11]:
from sklearn.preprocessing import OneHotEncoder
#instantiate one hot encoder
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_encoder.fit(cat_data)
cat_ohe = ohe_encoder.transform(cat_data)
cat_ohe

array([[0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [12]:
pd.DataFrame(cat_ohe, columns=ohe_encoder.get_feature_names(cat_data.columns))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
998,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
999,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1001,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [13]:
#instantiate column transformer
#give it the steps in the order to apply them.

from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

cat_selector = make_column_selector(dtype_include='object')
# make a numeric column selector
num_selector = make_column_selector(dtype_include='number')

median_imputer = SimpleImputer(strategy='median')
freq_imputer = SimpleImputer(strategy = 'most_frequent')

median_tuple = (median_imputer, num_selector)
#impute most frequent of categorical columns
freq_tuple = (freq_imputer, cat_selector)

In [14]:
# our new import
from sklearn.compose import make_column_transformer
#instantiate column transformer
#give it the steps in the order to apply them.
column_transformer = make_column_transformer(median_tuple, freq_tuple)

In [15]:
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('simpleimputer-1',
                                 SimpleImputer(strategy='median'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f0d96f7e590>),
                                ('simpleimputer-2',
                                 SimpleImputer(strategy='most_frequent'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f0d96f7e190>)])

In [16]:
X_train_processed = column_transformer.transform(X_train)
X_test_processed = column_transformer.transform(X_test)

In [17]:
X_train_processed

array([[24.0, 23.655, 0.0, 'male', 'no', 'northwest'],
       [28.0, 26.51, 2.0, 'female', 'no', 'southeast'],
       [51.0, 39.7, 1.0, 'male', 'no', 'southwest'],
       ...,
       [58.0, 25.175, 0.0, 'male', 'no', 'northeast'],
       [37.0, 47.6, 2.0, 'female', 'yes', 'southwest'],
       [55.0, 29.9, 0.0, 'male', 'no', 'southwest']], dtype=object)

In [18]:
# 5. Instantiate a linear regression model
from sklearn.linear_model import LinearRegression

In [19]:
reg = LinearRegression()

In [20]:
X_train.shape

(1003, 6)

In [21]:
X_test.shape

(335, 6)

In [22]:
# 6. Fit the model on the training data
reg.fit(X_train,y_train)

ValueError: ignored

In [ ]:
# 7. Evaluate the model performance on both the training set and the test set using R^2 score

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
train_score = reg.score(X_train, y_train)
print(train_score)

In [ ]:
test_score = reg.score(X_test, y_test)
print(test_score)